# Data Cleaning

### Import libraries

In [1]:
import pandas as pd
import numpy as np

Read in combined.csv file and labels file.

In [2]:
combined_df = pd.read_csv("./data/combined.csv")
label_df = pd.read_csv("./labels/part-00000-e9445087-aa0a-433b-a7f6-7f4c19d78ad6-c000.csv")

Since all duplicated IDs have mismatching labels, drop all duplicates from both label df and combined df.

In [3]:
dup_list = label_df[label_df.duplicated(subset=["bookingID"], keep='first')]["bookingID"].tolist()
label_df.drop_duplicates(subset=["bookingID"], keep=False, inplace=True)

In [4]:
for bookingID in dup_list:
    combined_df = combined_df[combined_df.bookingID != bookingID]
combined_df = filter_duplicate(combined_df)

Check Accuracy column and flag out IDs which contains values that lie outside the 99th percentile. For IDs that have more than 20% of their rows flagged as inaccurate, the ID will be dropped entirely. Else, only the inaccurate rows for the ID will be dropped.

In [5]:
low_acc_id = combined_df.loc[combined_df['Accuracy'] > combined_df["Accuracy"].quantile(0.99)].bookingID.value_counts().to_dict()
dropped_ids = []
for iteration, (bookingID, counts) in enumerate(low_acc_id.items()):
    if len(combined_df.loc[combined_df['bookingID'] == bookingID]) * 0.2 < counts:
        combined_df = combined_df[combined_df["bookingID"] != bookingID]
        dropped_ids.append(bookingID)
    else:
        combined_df.drop(combined_df[(combined_df['bookingID'] == bookingID) & (combined_df['Accuracy'] > combined_df["Accuracy"].quantile(0.99))].index, inplace=True)
    print("Checking Accuracy for {} IDs: {}/{}".format(len(low_acc_id), iteration+1, len(low_acc_id)), end='\r', flush=True)
print("\nTotal number of IDs fully dropped: {}".format(len(dropped_ids)))

Checking Accuracy for 2799 IDs: 2799/2799
Total number of IDs fully dropped: 236


Check Speed column and flag out IDs which contains values that are less than 0 (negative). For IDs that have more than 20% of their rows flagged as inaccurate, the ID will be dropped entirely. Else, only the inaccurate rows for the ID will be dropped.

In [6]:
low_acc_id = combined_df.loc[combined_df['Speed'] < 0].bookingID.value_counts().to_dict()
dropped_ids = []
for iteration, (bookingID, counts) in enumerate(low_acc_id.items()):
    if len(combined_df.loc[combined_df['bookingID'] == bookingID]) * 0.2 < counts:
        combined_df = combined_df[combined_df["bookingID"] != bookingID]
        dropped_ids.append(bookingID)
    else:
        combined_df.drop(combined_df[(combined_df['bookingID'] == bookingID) & (combined_df['Speed'] < 0)].index, inplace=True)
    print("Checking Speed for {} IDs: {}/{}".format(len(low_acc_id), iteration+1, len(low_acc_id)), end='\r', flush=True)
print("\nTotal number of IDs fully dropped: {}".format(len(dropped_ids)))

Checking Speed for 7510 IDs: 7510/7510
Total number of IDs fully dropped: 50


Save the filtered combine df to csv.

In [7]:
combined_df.to_csv("./data/final_filtered.csv", index=False)

Remove the dropped IDs from the label df as well and save to csv.

In [8]:
label_ids = label_df['bookingID'].tolist()
combined_ids = combined_df['bookingID'].unique().tolist()
dropped_ids = [x for x in label_ids if x not in combined_ids]
for ids in dropped_ids:
    label_df.drop(label_df[label_df['bookingID'] == ids].index, inplace=True)
label_df = label_df.sort_values(by=['bookingID'])
label_df.to_csv("./data/final_labels.csv", index=False)